In [18]:
from DeepEEG.input_preparation import build_inputs


import sys
from keras.layers import BatchNormalization
from keras.layers import MaxPooling1D, AveragePooling1D
from sklearn.ensemble import RandomForestClassifier
from pandas.core.frame import DataFrame
sys.path.insert(0, "/home/cirl/Amir/Human-Activity-EEG-Accelerometer")
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
import time

import tensorflow as tf
import random as rn
from keras import backend as K, optimizers
from DeepEEG.input_preparation import build_inputs
from keras.callbacks import EarlyStopping, CSVLogger
from DeepEEG.evaluation import compute_accuracy, evalRes
from keras.utils import plot_model
from keras.layers import LSTM
from keras.layers import Conv2D, Conv1D, SeparableConv1D
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
# https://pdfs.semanticscholar.org/df0b/05d8985846e694cda62d41a04e7c85090fa6.pdf

rn.seed(12345)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(3)
rn.seed(12345)

classes = 3




In [19]:
X_train, y_train, X_test, y_test, train_labels, test_labels = build_inputs(False, 330)


In [20]:
X_train

array([[[0.1417911 , 0.77868844],
        [0.29104478, 0.65573775],
        [0.22388067, 0.83606564],
        ...,
        [0.5746267 , 0.45081972],
        [0.67910431, 0.28688536],
        [0.72388067, 0.29508197]],

       [[0.32089541, 0.67213128],
        [0.3880598 , 0.59836056],
        [0.40298498, 0.59016394],
        ...,
        [0.73880584, 0.61475408],
        [0.73880584, 0.59836056],
        [0.32089541, 0.22131156]],

       [[0.19402976, 0.37704931],
        [0.19402976, 0.40983606],
        [0.19402976, 0.40983606],
        ...,
        [0.50746259, 0.78688536],
        [0.37313435, 0.78688536],
        [0.35074631, 0.69672142]],

       ...,

       [[0.66666667, 0.7368418 ],
        [0.78431437, 0.55263184],
        [0.37254923, 0.92105273],
        ...,
        [0.78431437, 0.52631543],
        [0.96078483, 0.57894727],
        [0.56862801, 0.60526269]],

       [[0.52941211, 0.42105273],
        [0.58823559, 0.44736816],
        [0.56862801, 0.3684209 ],
        .

In [21]:
epochs = 50  # 21


In [22]:
def build_model(X_train, row, cell):
    model = Sequential()
    model.add(Conv1D(32, 4, strides=1, data_format='channels_last',
             input_shape=(330, 2)))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Conv1D(64, 8, strides=4, data_format='channels_last'))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=16, strides = 2, padding='valid'))
    model.add(Dense(512))
    model.add(Activation("tanh"))
    model.add(LSTM(128, activation='tanh', recurrent_activation='hard_sigmoid', \
                    use_bias=True, kernel_initializer='glorot_uniform', \
                    recurrent_initializer='orthogonal', \
                    unit_forget_bias=True, kernel_regularizer=None, \
                    recurrent_regularizer=None, \
                    bias_regularizer=None, activity_regularizer=None, \
                    kernel_constraint=None, recurrent_constraint=None, \
                    bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, \
                    implementation=1, return_sequences=True, return_state=False, \
                    go_backwards=False, stateful=False, unroll=False))
    model.add(Dropout(0.4))
    model.add(LSTM(128, activation='tanh', recurrent_activation='hard_sigmoid', \
                    use_bias=True, kernel_initializer='glorot_uniform', \
                    recurrent_initializer='orthogonal', \
                    unit_forget_bias=True, kernel_regularizer=None, \
                    recurrent_regularizer=None, \
                    bias_regularizer=None, activity_regularizer=None, \
                    kernel_constraint=None, recurrent_constraint=None, \
                    bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, \
                    implementation=1, return_sequences=False, return_state=False, \
                    go_backwards=False, stateful=False, unroll=False))
    model.add(Dropout(0.4))
    model.add(Dense(3, activation="softmax"))
    opt = optimizers.adam(lr=0.001)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    plot_model(model, to_file='model.png', show_shapes=True)
    model.summary()
    return model

In [23]:
model = build_model(X_train, 0, 0)


2024-01-10 07:53:35.591638: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-10 07:53:35.591678: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: LD-IdeaPad-Gaming-3-15IHU6
2024-01-10 07:53:35.591681: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: LD-IdeaPad-Gaming-3-15IHU6
2024-01-10 07:53:35.591912: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 525.147.5
2024-01-10 07:53:35.591926: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 525.147.5
2024-01-10 07:53:35.591928: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 525.147.5


AttributeError: module 'keras.optimizers' has no attribute 'adam'